In [ ]:
# env = myenv

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
from langchain_classic.chains import LLMChain, SequentialChain

In [ ]:
from langchain_community.callbacks import get_openai_callback

In [ ]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from pypdf import PdfReader


In [ ]:
os.environ.pop("OPENROUTER_API_KEY", None)
print(os.getenv("OPENROUTER_API_KEY"))

In [ ]:
load_dotenv(override=True)

In [ ]:
key=os.getenv("Openai_api_key")

In [ ]:
print(key)

In [ ]:
llm=ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    openai_api_key=key,
    tiktoken_model_name='openai/gpt-oss-20b:free',
    temperature=0.7
    )

In [ ]:
llm

In [ ]:
with open(r"C:\Users\user\OneDrive\Desktop\Generative AI\genai-github\response.json", "r") as f:
    RESPONSE_JSON = json.load(f)

In [ ]:
template = """
Text:
{text}

You are an expert MCQ generator.

Your task is to create exactly {number} multiple-choice questions (MCQs) 
for {subject} students using the above text.

Difficulty / Tone:
{tone}

Instructions:
- Each question must be clear and relevant to the given text.
- Do NOT repeat any questions.
- Each MCQ must have four options.
- Only one option should be correct.
- Ensure all questions are conceptually accurate.

Return the output strictly in the JSON format shown below.
Do NOT add any extra text outside the JSON.

### RESPONSE_JSON
{RESPONSE_JSON}
"""


In [ ]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','RESPONSE_JSON'],
    template=template
)

In [ ]:
quiz_chain=LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key='quiz',
    verbose=True
    )

In [ ]:
template2 = """
You are an expert educational content reviewer.

Below is a quiz for {subject} students.

Your task is to:
1. Verify the factual correctness of each MCQ.
2. Confirm that the provided correct answers are accurate.
3. Improve grammar, clarity, and wording where needed.
4. Ensure all questions meet proper academic standards.
5. Fix or upgrade any MCQ that has errors or ambiguity.
6. Do NOT change the total number of questions.
7. Follow the response structure exactly as defined in response.json.

### QUIZ DATA
{quiz}
"""


In [ ]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=template2
)

In [ ]:
review_chain=LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key='review',
    verbose=True
    )

In [ ]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=['text', 'number', 'subject','RESPONSE_JSON', 'tone'],  # combined input vars
    output_variables=['quiz', 'review'],                    # plural
    verbose=True
)

In [ ]:
path = r"C:\Users\user\OneDrive\Desktop\Generative AI\genai-github\data.txt"

In [ ]:
with open(path,'r') as file:
    Text=file.read()

In [ ]:
Text,
number=5,
subject='AI',
tone='simple',
RESPONSE_JSON=RESPONSE_JSON

In [ ]:
inputs = {
    "text": Text,
    "number": number,
    "subject": subject,
    "tone": tone,
    "RESPONSE_JSON":json.dumps(RESPONSE_JSON)
}
with get_openai_callback() as cb:
    response = generate_evaluate_chain(inputs)  # Direct call


In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz=response.get("quiz")

In [ ]:
quiz=json.loads(quiz)

In [ ]:
quiz

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['mcq']
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choice":options,"Correct":correct})

In [ ]:
df=pd.DataFrame(quiz_table_data)

In [ ]:
df.to_csv("genaiquiz1.csv")